# Analyzing fire payrolls in L.A.: 2011-2019

This notebook processess government payroll [data](https://publicpay.ca.gov/Reports/RawExport.aspx) compiled and released annually by the California state controller's office. The data include anonymized salary information for all employees at cities, counties, special districts and state government. 

---

### Load python tools

In [36]:
import altair as alt
import altair_latimes as lat
import pandas as pd
import geopandas as gpd
import cpi
from urllib.request import urlopen 
import pyarrow
import jenkspy
import matplotlib.pyplot as plt
%matplotlib inline
import json
import numpy as np
from altair import datum
import altair_latimes as lat
alt.themes.register('latimes', lat.theme)
alt.themes.enable('latimes')
pd.options.display.float_format = '{:,.0f}'.format

### Import fire payroll data

In [37]:
# processed in 01-california-payroll-descriptives
agency_ot = pd.read_csv('output/agency_overtime.csv')
agency_ot_share = pd.read_csv('output/agency_overtime_share_ot.csv')
payroll_fire = pd.read_csv('/Users/mhustiles/data/data/controller/output/payroll_fire.csv')

---

### Filter dataframe to include only Los Angeles city and county fire

In [38]:
la = pd.DataFrame(payroll_fire[payroll_fire['employer'] == 'LOS ANGELES'])

In [39]:
la.employerfull = la.employerfull.str.replace('_', ' ')

In [40]:
la_agencies = la.groupby(['employerfull', 'year', 'type'])\
    .agg({'adjusted_overtime':'sum'}).reset_index()

In [41]:
la_agencies

,employerfull,year,type,adjusted_overtime
0,LOS ANGELES CITY,2011,CITY,"109,199,314"
1,LOS ANGELES CITY,2012,CITY,"125,749,783"
2,LOS ANGELES CITY,2013,CITY,"154,272,930"
3,LOS ANGELES CITY,2014,CITY,"174,201,837"
4,LOS ANGELES CITY,2015,CITY,"199,022,322"
5,LOS ANGELES CITY,2016,CITY,"198,223,610"
6,LOS ANGELES CITY,2017,CITY,"204,805,434"
7,LOS ANGELES CITY,2018,CITY,"198,411,726"
8,LOS ANGELES CITY,2019,CITY,"207,856,755"
9,LOS ANGELES COUNTY,2011,COUNTY,"143,138,158"


In [42]:
alt.Chart(la_agencies).mark_line().encode(
    x='year:O',
    y='adjusted_overtime',
    color='employerfull'
)

alt.Chart(...)

In [43]:
la_agencies.to_csv('output/la_agencies.csv')

---

### Create a dataframe combining and pivoting listing all the large places

In [44]:
la_pivot_ot = pd.DataFrame(pd.pivot_table(la_agencies, \
                values='adjusted_overtime', index=['employerfull', 'type'], columns=['year']).reset_index().fillna(0))

In [45]:
la_pivot_ot.columns = la_pivot_ot.columns.map(str)

### What did each fire department pay in overtime (just to firefighter titles) each year? 

In [46]:
la_pivot_ot['ot_pct_change'] = \
                ((la_pivot_ot['2018']-la_pivot_ot['2011'])/\
                 la_pivot_ot['2011'])*100

In [47]:
la_pivot_ot.head()

year,employerfull,type,2011,2012,2013,2014,2015,2016,2017,2018,2019,ot_pct_change
0,LOS ANGELES CITY,CITY,"109,199,314","125,749,783","154,272,930","174,201,837","199,022,322","198,223,610","204,805,434","198,411,726","207,856,755",82
1,LOS ANGELES COUNTY,COUNTY,"143,138,158","147,633,267","162,505,150","165,835,905","178,752,293","200,750,586","206,097,766","203,838,869","206,244,625",42


In [48]:
#2011
'${:,.0f}'.format((la_pivot_ot.iloc[1,2] + la_pivot_ot.iloc[0,2]))

'$252,337,472'

In [49]:
#2018
'${:,.0f}'.format((la_pivot_ot.iloc[1,9] + la_pivot_ot.iloc[0,9]))

'$402,250,595'

In [50]:
'{:,.2f}%'.format(((la_pivot_ot.iloc[1,9] + la_pivot_ot.iloc[0,9])-(la_pivot_ot.iloc[1,2] + la_pivot_ot.iloc[0,2]))\
        / (la_pivot_ot.iloc[1,2] + la_pivot_ot.iloc[0,2])*100)

'59.41%'

In [51]:
la_pivot_ot.to_csv('output/la_city_county.csv')

---

### Charting firefighter wages and benefits in Los Angeles

In [52]:
la_all = la.groupby(['year', 'employerfull']).agg({'adjusted_overtime':'sum', \
                                            'adjusted_basewages':'sum', \
                                        'adjusted_wages':'sum', \
                                            'adjusted_benefits':'sum'\
                                            }).reset_index()

### Clean up the column names

In [53]:
la_all.rename(columns={'adjusted_overtime': 'Overtime',\
                    'adjusted_basewages': 'BaseWages',\
                   'adjusted_wages': 'Wages',\
                   'adjusted_benefits':'Benefits',\
                     'year': 'Year',
                      'employerfull': 'Place'}, inplace=True)

### Melt the dataframe for charting

In [54]:
la_melt = pd.melt(la_all,id_vars=['Year', 'Place'], value_vars=['Overtime','BaseWages'],\
        var_name='Compensation', value_name='Total')

### Chart the overtime amount in annual dollars

In [71]:
otdollars = alt.Chart(la_melt).mark_area().encode(
    x=alt.X("Year:N", title=' '),
    y=alt.Y("Total:Q", title=' ', axis=alt.Axis(tickCount=6, format='$,s')),
    color=alt.Color("Compensation:N", title=''),
    order=alt.Order(
      # Sort the segments of the bars by this field
      'Compensation',
      sort='ascending'
    ),
    facet=alt.Facet('Place:N', title='Overtime in inflation-adjusted dollars')
).properties(width=300, height=300, title='Fireighter pay')

otdollars.configure_header(
    titleColor='gray',
    titleFontSize=14,
    titleFontWeight = 'normal',
    titleOrient = 'top',
    titleAnchor='middle',
    titlePadding=10,
    labelFontSize=11
).configure_legend(
    orient='right'
)

alt.Chart(...)

In [56]:
la_melt.sort_values(by=['Place', 'Compensation', 'Year'], ascending=True)

,Year,Place,Compensation,Total
18,2011,LOS ANGELES CITY,BaseWages,"404,817,544"
20,2012,LOS ANGELES CITY,BaseWages,"385,790,295"
22,2013,LOS ANGELES CITY,BaseWages,"380,190,626"
24,2014,LOS ANGELES CITY,BaseWages,"340,290,009"
26,2015,LOS ANGELES CITY,BaseWages,"383,396,417"
28,2016,LOS ANGELES CITY,BaseWages,"385,572,696"
30,2017,LOS ANGELES CITY,BaseWages,"407,234,140"
32,2018,LOS ANGELES CITY,BaseWages,"408,273,826"
34,2019,LOS ANGELES CITY,BaseWages,"420,685,996"
0,2011,LOS ANGELES CITY,Overtime,"109,199,314"


### Normalized version

In [72]:
otshare = alt.Chart(la_melt).mark_area().encode(
    x=alt.X("Year:N", title=' '),
    y=alt.Y("Total:Q", stack="normalize", title=' ', axis=alt.Axis(tickCount=6, format='%')),
    color="Compensation:N",
    order=alt.Order(
      # Sort the segments of the bars by this field
      'Compensation',
      sort='ascending'
    ),
    facet=alt.Facet('Place:N', title=' ')
).properties(width=300, height=300, title='Firefighter compensation in Los Angeles')

otshare.configure_header(
    titleColor='gray',
    titleFontSize=14,
    titleFontWeight = 'normal',
    titleOrient = 'top',
    titleAnchor='middle',
    titlePadding=10,
    labelFontSize=11
).configure_legend(
    orient='right'
)

alt.Chart(...)

In [58]:
la_melt.sort_values(by=['Place', 'Compensation']).to_csv('output/la_melt.csv')

---

### How many firefighters received six-figure amounts of overtime in a year?

In [59]:
la_highest = la[la['overtime'] > 200000]

In [60]:
la_high = la[la['overtime'] > 100000]

In [61]:
print('$200k:',len(la_highest),'| $100k:',len(la_high)) 

$200k: 203 | $100k: 6274


### Group by agency type and year and count recipients of $100k overtime

In [62]:
la_high_years = la_high.groupby(['year','type']).agg('size').reset_index(name='count')
la_high_years.sort_values(by='type', ascending=False).head(25)

,year,type,count
9,2015,COUNTY,408
7,2014,COUNTY,220
15,2018,COUNTY,611
13,2017,COUNTY,631
11,2016,COUNTY,577
1,2011,COUNTY,20
17,2019,COUNTY,699
5,2013,COUNTY,161
3,2012,COUNTY,58
6,2014,CITY,282


### Group by agency type and year and count recipients of $200k overtime

In [63]:
la_highest_years = la_highest.groupby(['year','type']).agg('size').reset_index(name='count')
la_highest_years.sort_values(by='type', ascending=False).head(25)

,year,type,count
4,2016,COUNTY,19
6,2017,COUNTY,25
8,2018,COUNTY,20
10,2019,COUNTY,8
0,2013,CITY,5
1,2014,CITY,6
2,2015,CITY,19
3,2016,CITY,22
5,2017,CITY,26
7,2018,CITY,23


### Group by agency type and year and count recipients of $200k overtime 

In [64]:
la_highest_years = la_highest.groupby(['year','type']).agg('size').reset_index(name='count')

In [73]:
alt.Chart(la_highest_years).mark_bar().encode(
    x=alt.X("year:N", title=' '),
    y=alt.Y("mean(count):Q", title=' ', axis=alt.Axis(tickCount=6, format='')),
    facet='type:O'
).properties(width=300, height=300, title='L.A. firefighters > $200k OT')

alt.Chart(...)

In [66]:
la_highest_years.sort_values(by=['type','year'], ascending=False).head(20)

,year,type,count
10,2019,COUNTY,8
8,2018,COUNTY,20
6,2017,COUNTY,25
4,2016,COUNTY,19
9,2019,CITY,30
7,2018,CITY,23
5,2017,CITY,26
3,2016,CITY,22
2,2015,CITY,19
1,2014,CITY,6


In [67]:
la_high_positions = la_high.groupby('position').agg({'adjusted_overtime':'sum', \
    'adjusted_basewages':'sum', \
    'adjusted_wages':'sum', \
    'adjusted_benefits':'sum'\
}).reset_index()

### Payroll by title

In [68]:
la_high_positions.sort_values(by='adjusted_overtime', ascending=False)

,position,adjusted_overtime,adjusted_basewages,adjusted_wages,adjusted_benefits
5,FIRE CAPTAIN (56 HOUR),"213,221,082","223,308,138","436,529,220","94,742,026"
22,FIREFIGHTER III,"120,791,345","110,353,192","231,144,538","33,111,885"
6,FIRE CAPTAIN I,"102,608,826","103,111,760","205,720,586","27,213,408"
10,FIRE FIGHTER (56 HOUR),"101,430,816","95,881,406","197,312,222","41,876,726"
11,FIRE FIGHTER SPECIALIST (56 HOUR),"99,758,637","97,376,676","197,135,313","42,468,462"
7,FIRE CAPTAIN II,"47,230,723","49,775,424","97,006,147","13,817,643"
1,BATTALION CHIEF/56 HOURS/,"38,676,046","50,255,737","88,931,783","23,772,214"
2,ENGINEER OF FIRE DEPARTMENT,"34,849,706","32,747,548","67,597,255","8,545,489"
16,FIRE INSPECTOR I,"20,380,203","19,111,266","39,491,469","6,209,887"
4,FIRE BATTALION CHIEF,"19,338,689","25,735,625","45,074,314","6,317,907"


---

### $300,000 salaries

In [69]:
threehundred = payroll_fire[(payroll_fire.adjusted_wages > 300000) &\
                            (payroll_fire.year == 2011) &\
                            (payroll_fire.employer.str.contains('LOS ANGELES'))]

In [70]:
threehundred.head()

,Unnamed: 0,index,year,type,population,employer,department,position,overtime,wages,benefits,employercounty,basewages,adjusted_overtime,adjusted_wages,adjusted_benefits,adjusted_basewages,employerfull
218606,8167026,8167026,2011,COUNTY,"9,889,520",LOS ANGELES,FIRE DEPARTMENT,BATTALION CHIEF/56 HOURS/,"99,228","267,607","60,417",LOS ANGELES,"168,379","112,779","304,152","68,668","191,373",LOS_ANGELES_COUNTY
218717,8167206,8167206,2011,COUNTY,"9,889,520",LOS ANGELES,FIRE DEPARTMENT,COUNTY FORESTER & FIRE WARDEN,0,"317,126","105,478",LOS ANGELES,"317,126",0,"360,433","119,882","360,433",LOS_ANGELES_COUNTY
237104,9032380,9032380,2011,CITY,"3,827,172",LOS ANGELES,LOS ANGELES FIRE DEPARTMENT,FIRE BATTALION CHIEF,"106,088","267,124","13,368",LOS ANGELES,"161,036","120,576","303,603","15,194","183,027",LOS_ANGELES_CITY
237378,9032654,9032654,2011,CITY,"3,827,172",LOS ANGELES,LOS ANGELES FIRE DEPARTMENT,FIRE CAPTAIN L,"148,172","270,622","13,368",LOS ANGELES,"122,450","168,407","307,579","15,194","139,172",LOS_ANGELES_CITY
237768,9033044,9033044,2011,CITY,"3,827,172",LOS ANGELES,LOS ANGELES FIRE DEPARTMENT,FIRE DEPUTY CHIEF,"28,546","294,742","13,368",LOS ANGELES,"266,196","32,444","334,992","15,194","302,548",LOS_ANGELES_CITY


---

Data source: https://publicpay.ca.gov/Reports/RawExport.aspx